# Active automata learning with LearnLib

This is an example of active automata learning with LearnLib from python using py4j. The code is based on https://github.com/mtf90/learnlib-py4j-example.

First, we define the system under learning (SUL). This is a python implementation of SUL interface of LearnLib in Java.

In [1]:
from py4j.java_gateway import JavaGateway, CallbackServerParameters

# This class represents the Python side implementation of our system under learning (SUL)
class PySUL:

    def __init__(self, gateway):
        self.alphabet = gateway.jvm.net.automatalib.words.impl.Alphabets.characters('a', 'b')
        self.state = 0

    def pre(self):
        self.state = 0

    def post(self): pass

    def step(self, sulInput):
        if sulInput == 'b':
            self.state = (self.state + 1) % 2
            return 'b'

        return '0' if self.state == 0 else '1'

    def canFork(self):
        return False

    def fork(self): pass

    class Java:
        implements = ["de.learnlib.api.SUL"]

Then, we connect to the JVM process.

In [2]:
gateway = JavaGateway(callback_server_parameters=CallbackServerParameters())

Here is the main learning part. We use TTT algorithm for active automata learning with W method as the equivalence oracle.

In [3]:
# Instantiate our Python SUL and get the learning alphabet
sul = PySUL(gateway)
alphabet = sul.alphabet

# Construct the membership oracle
mq_oracle = gateway.jvm.de.learnlib.oracle.membership.SULOracle(sul)

# Construct the equivalence oracle
eq_oracle = gateway.jvm.de.learnlib.oracle.equivalence.WMethodEQOracle(mq_oracle, 3)

# Construct the learning algorithm (here TTT)
ttt = gateway.jvm.de.learnlib.algorithms.ttt.mealy.TTTLearnerMealyBuilder() \
    .withAlphabet(alphabet) \
    .withOracle(mq_oracle) \
    .create()

# Construct the experiment, that runs the active learning loop until no more counterexamples can be found.
experiment = gateway.jvm.de.learnlib.util.Experiment(ttt, eq_oracle, alphabet)
experiment.run()

# Get the final hypothesis of our SUL
hyp = experiment.getFinalHypothesis()

After the learning, we obtain the learned automaton in DOT format.

In [4]:
# Construct a buffer that we will use to print results on the Python side of our setup
string_writer = gateway.jvm.java.io.StringWriter()

# Serialize the hypothesis to the DOT format and write it to the string_writer
gateway.jvm.net.automatalib.serialization.dot.GraphDOT.write(hyp, alphabet, string_writer,
                                                              # While varargs allow us to skip this parameter in Java, the method signature expects an array \
                                                              gateway.new_array(
                                                                  gateway.jvm.net.automatalib.serialization.dot.DOTVisualizationHelper,
                                                                  0))

print("Learned model in DOT format:")
print()
print(string_writer.toString())

Learned model in DOT format:

digraph g {

	s0 [shape="circle" label="s0"];
	s1 [shape="circle" label="s1"];
	s0 -> s0 [label="a / 0"];
	s0 -> s1 [label="b / b"];
	s1 -> s1 [label="a / 1"];
	s1 -> s0 [label="b / b"];

__start0 [label="" shape="none" width="0" height="0"];
__start0 -> s0;
}



We can also directly show the states and transitions

In [5]:
hyp.getStates()

[JavaObject id=o516, JavaObject id=o517]

In [6]:
for state in hyp.getStates():
    print(state)

s0
s1


In [7]:
sul.alphabet

['a', 'b']

In [8]:
# State: http://learnlib.github.io/learnlib/maven-site/0.13.0/apidocs/de/learnlib/algorithms/ttt/base/TTTState.html
for srcID in range(hyp.getStates().size()):
    print(f'Prefix to {hyp.getStates()[srcID]}: {hyp.getStates()[srcID].getAccessSequence()}')
    for charID in range(hyp.getInputAlphabet().size()):
        transition = hyp.getTransition(srcID, charID)
        dst = hyp.getSuccessor(transition)
        print(f'{hyp.getStates()[srcID]} --{alphabet[charID]}/{hyp.getTransitionOutput(transition)}--> {dst}')

Prefix to s0: ε
s0 --a/0--> s0
s0 --b/b--> s1
Prefix to s1: b
s1 --a/1--> s1
s1 --b/b--> s0


Finally, we close the connection to the JVM.

In [9]:
# Close our connection
gateway.close()